In [19]:
# Activate the environment using the path to the Macro repository
macro_repo_path = dirname(@__DIR__)
using Pkg; Pkg.activate(macro_repo_path)

  Activating project at `~/Code/Macro`


In [20]:
using Macro
using HiGHS
using CSV
using DataFrames
using JSON3

In this tutorial, we start from a single zone electricity system with four resource clusters: utility scale solar PV, land-based wind power generation, natural gas combined cycle power plants, and electricity storage. 

We consider three commodities: electricity, natural gas, and $\text{CO}_2$. 

Initially, hydrogen is modeled exogenously, including a fixed hydrogen demand as part of the electricity demand. This is equivalent to assuming the existance of an electrolyzer that continuously consumes electricity to meet the hydrogen demand.

We model a greenfield scenario with a carbon price of 200 $/ton.

***Note: We use the default units in MACRO: MWh for energy vectors, metric tons for other commodities (e.g., $\text{CO}_2$) and dollars for costs***

We first load the inputs:

In [21]:
system = Macro.load_system("one_zone_electricity_only");

┌ Info: Loading JSON data from /Users/fpecci/Code/Macro/tutorials/one_zone_electricity_only/system_data.json
└ @ Macro /Users/fpecci/Code/Macro/src/load_inputs/load_macroobject.jl:135
┌ Info: Loading JSON data from one_zone_electricity_only/system_data.json
└ @ Macro /Users/fpecci/Code/Macro/src/load_inputs/load_macroobject.jl:135
┌ Info: Loading JSON data from one_zone_electricity_only/system/nodes.json
└ @ Macro /Users/fpecci/Code/Macro/src/load_inputs/load_macroobject.jl:135
┌ Info: Loading columns [:natgas_SE] from CSV data from one_zone_electricity_only/system/fuel_prices.csv
└ @ Macro /Users/fpecci/Code/Macro/src/load_inputs/file_io/csv.jl:8
┌ Info: Loading CSV data from one_zone_electricity_only/system/fuel_prices.csv
└ @ Macro /Users/fpecci/Code/Macro/src/load_inputs/file_io/csv.jl:14
┌ Info: Loading columns [:Demand_MW_z1] from CSV data from one_zone_electricity_only/system/demand.csv
└ @ Macro /Users/fpecci/Code/Macro/src/load_inputs/file_io/csv.jl:8
┌ Info: Loading CSV data 

system.settings.Scaling = false


We are now ready to generate the MACRO capacity expansion model. Because MACRO is designed to be solved by [high performance decomposition algorithms](https://arxiv.org/abs/2403.02559), the model formulation has a specific block structure that can be exploited by these schemes. In the case of 3 operational sub-periods, the block structure looks like this:

![model_structure](images/model_structure.png)

In [22]:
model = Macro.generate_model(system)

┌ Info: Starting model generation
└ @ Macro /Users/fpecci/Code/Macro/src/generate_model.jl:3
┌ Info: Adding linking variables
└ @ Macro /Users/fpecci/Code/Macro/src/generate_model.jl:15
┌ Info: Defining available capacity
└ @ Macro /Users/fpecci/Code/Macro/src/generate_model.jl:18
┌ Info: Generating planning model
└ @ Macro /Users/fpecci/Code/Macro/src/generate_model.jl:21
┌ Info: Generating operational model
└ @ Macro /Users/fpecci/Code/Macro/src/generate_model.jl:24
┌ Info: Model generation complete, it took 1.4618639945983887 seconds
└ @ Macro /Users/fpecci/Code/Macro/src/generate_model.jl:29


A JuMP Model
├ solver: none
├ objective_sense: MIN_SENSE
│ └ objective_function_type: JuMP.AffExpr
├ num_variables: 96373
├ num_constraints: 271581
│ ├ JuMP.AffExpr in MOI.EqualTo{Float64}: 43801
│ ├ JuMP.AffExpr in MOI.GreaterThan{Float64}: 26281
│ ├ JuMP.AffExpr in MOI.LessThan{Float64}: 105127
│ ├ JuMP.VariableRef in MOI.EqualTo{Float64}: 5
│ └ JuMP.VariableRef in MOI.GreaterThan{Float64}: 96367
└ Names registered in the model
  └ :eFixedCost, :eVariableCost, :vREF

Next, we set the optimizer. Note that we are using the open-source LP solver [HiGHS](https://highs.dev/), alternatives include the commerical solvers [Gurobi](https://www.gurobi.com/), [CPLEX](https://www.ibm.com/products/ilog-cplex-optimization-studio), [COPT](https://www.copt.de/).

In [23]:
Macro.set_optimizer(model, HiGHS.Optimizer);

Finally, we solve the capacity expansion model:

In [24]:
Macro.optimize!(model)

Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [7e-08, 3e+05]
  Cost   [1e-01, 6e+07]
  Bound  [1e+00, 1e+00]
  RHS    [0e+00, 0e+00]
Presolving model
162466 rows, 83629 cols, 504916 nonzeros  0s
144862 rows, 66025 cols, 514072 nonzeros  0s
Presolve : Reductions: rows 144862(-30347); columns 66025(-30348); elements 514072(-108712)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -1.0020190497e+07 Ph1: 28379(68837.6); Du: 7952(1.00202e+07) 0s
      39907     8.0956077523e+10 Pr: 16955(2.87562e+09) 5s
      57165     8.9445352349e+10 Pr: 48320(1.56818e+11) 11s
      70861     9.4485764481e+10 Pr: 17360(8.697e+08) 16s
      84493     9.6905286410e+10 Pr: 3254(2.40533e+07); Du: 0(3.7078e-08) 21s
      90658     9.7171882459e+10 Pr: 0(0); Du: 0(6.0944e-10) 23s
      90658     9.7171882459e+10 Pr: 0(0); Du: 0(6.0944e-10) 23s
Solving

And extract the results:

In [25]:
capacity_results = Macro.get_optimal_asset_capacity(system)

Row,asset,type,capacity,additions,retirements
,Symbol,Symbol,Float64,Float64,Float64
1,battery_SE,Battery,1.1172e5,1.1172e5,0.0
2,SE_naturalgas_ccavgcf_moderate_0,ThermalPower{NaturalGas},1.26487e5,220.746,0.0
3,SE_utilitypv_class1_moderate_70_0_2_1,VRE,3.46851e5,3.46851e5,0.0
4,SE_landbasedwind_class4_moderate_70_1,VRE,2.63429e5,2.63429e5,0.0


The total system cost (in dollars) is:

In [26]:
Macro.objective_value(model)

9.717188245934773e10

and the total emissions (in metric tonnes) are:

In [27]:
co2_node_idx = findfirst(isa.(system.locations,Node{CO2}).==1)
Macro.value(sum(system.locations[co2_node_idx].operation_expr[:emissions]))

7.947205644186783e7

<!-- **Task :** Set a strict net-zero $\text{CO}_2$ cap by removing the slack allowing constraint violation for a penalty. This can be done by deleting the field `price_unmet_policy` from the $\text{CO}_2$ node in file `one_zone_electricity_only/system/nodes.json`

Re-load the model with the new inputs and solve it to obtain the capacity results. Is there any difference? -->